In [57]:
import requests
import pandas as pd
import json

url = "https://spotify23.p.rapidapi.com/playlist_tracks/"
headers = {
	"X-RapidAPI-Key": "fd4c646022msh563de5bf4fb79e0p1a159cjsn409672e9fc1b",
	"X-RapidAPI-Host": "spotify23.p.rapidapi.com"
}

playlist_ids = ["37i9dQZF1DWUZv12GM5cFk", "37i9dQZF1DX9Ol4tZWPH6V", "37i9dQZF1DX0P7PzzKwEKl", "37i9dQZF1DXaW8fzPh9b08", "37i9dQZF1DWTWdbR13PQYH", 
                "37i9dQZF1DWWzQTBs5BHX9", "37i9dQZF1DX1vSJnMeoy3V", "37i9dQZF1DX3j9EYdzv2N9", "37i9dQZF1DWYuGZUE4XQXm", "37i9dQZF1DX4UkKv8ED8jp", 
                "37i9dQZF1DXc6IFF23C9jj", "37i9dQZF1DXcagnSNtrGuJ", "37i9dQZF1DX0yEZaMOXna3", "37i9dQZF1DX3Sp0P28SIer", "37i9dQZF1DX0h0QnLkMBl4", 
                "37i9dQZF1DX9ukdrXQLJGZ", "37i9dQZF1DX8XZ6AUo9R4R", "37i9dQZF1DWTE7dVUebpUW", "37i9dQZF1DXe2bobNYDtW8", "37i9dQZF1DWVRSukIED0e9"]

f = open('top_100_from_2000_to_2019.csv', 'a')
f.write('id,year,top_100_rank,track_name,artist,explicit,duration_ms,popularity,album_name,album_type,release_date,num_tracks_in_album\n')

artist_ids = {}
id = 0
year = 1999

for playlist in playlist_ids:
    querystring = {"id":playlist,"offset":"0","limit":"100"}
    response = requests.request("GET", url, headers=headers, params=querystring)

    year += 1
    rank = 0

    for item in json.loads(response.text)['items']:
        id += 1
        rank += 1
        name = item['track']['name']
        artist = item['track']['artists'][0]['name']
        if item['track']['artists'][0]['id'] not in artist_ids:
            artist_ids[item['track']['artists'][0]['id']] = artist
        explicit = item['track']['explicit']
        duration_ms = item['track']['duration_ms']
        popularity = item['track']['popularity']
        album = item['track']['album']['name']
        album_type = item['track']['album']['album_type']
        release_date = item['track']['album']['release_date']
        num_tracks = item['track']['album']['total_tracks']
        f.write("{},{},{},\"{}\",\"{}\",{},{},{},\"{}\",{},{},{}\n".format(id, year, rank, name, artist, explicit, duration_ms, popularity, 
                                                                           album.replace('"', ''), album_type, release_date, num_tracks))
f.close()

# saving spotify artist ids to gather info like artist genres if we have time
f = open('artist_ids.csv', 'a')
f.write('artist,id\n')
for key in artist_ids:
    f.write('{},{}\n'.format(artist_ids[key], key))
f.close()

In [58]:
df = pd.read_csv('top_100_from_2000_to_2019.csv')
df

,id,year,top_100_rank,track_name,artist,explicit,duration_ms,popularity,album_name,album_type,release_date,num_tracks_in_album
0,1,2000,1,Oops!...I Did It Again,Britney Spears,False,211160,74,Oops!... I Did It Again,album,2000-05-16,12
1,2,2000,2,All The Small Things,blink-182,False,167066,77,Enema Of The State,album,1999-06-01,12
2,3,2000,3,Breathe,Faith Hill,False,250546,63,Breathe,album,1999-11-09,13
3,4,2000,4,It's My Life,Bon Jovi,False,224493,76,Crush,album,2000-01-01,12
4,5,2000,5,Bye Bye Bye,*NSYNC,False,200400,67,No Strings Attached,album,2000-03-21,12
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,2019,96,Motivation,Normani,False,193837,66,Motivation,single,2019-08-16,1
1996,1997,2019,97,Lost Without You,Freya Ridings,False,225853,65,Freya Ridings,album,2019-07-19,12
1997,1998,2019,98,Ladbroke Grove,AJ Tracey,False,190537,65,AJ Tracey,album,2019-02-08,15
1998,1999,2019,99,Clout (feat. Cardi B),Offset,True,205803,65,FATHER OF 4,album,2019-02-22,16
